In [16]:
import pandas as pd
import numpy as np

# Read in Data

In [77]:
# specify states for bridges
states = {'DE': 10, 'MD': 24, 'PA': 42}

import os
os.listdir("data/nbi/2017del/")

['MD17.txt',
 'SC17.txt',
 'CT17.txt',
 'LA17.txt',
 'TX17.txt',
 'TN17.txt',
 'PA17.txt',
 'KY17.txt',
 'NV17.txt',
 'WV.txt',
 'OR17.txt',
 'VA17.txt',
 'AR17.txt',
 'AL17.txt',
 'KS17.txt',
 'IL17.txt',
 'MO17.txt',
 'RI17.txt',
 'NJ17.txt',
 'MA17.txt',
 'OH17.txt',
 'NC17.txt',
 'VT17.txt',
 'ND17.txt',
 'MI17.txt',
 'CO17.txt',
 'CA17.txt',
 'MN17.txt',
 'GA17.txt',
 'SD17.txt',
 'NH17.txt',
 'AZ17.txt',
 'ME17.txt',
 'NE17.txt',
 'WA17.txt',
 'FL17.txt',
 'DE17.txt',
 'NM17.txt',
 'MT17.txt',
 'IA17.txt',
 'WI17.txt',
 'IN17.txt',
 'WY17.txt',
 'PR17.txt',
 'HI17.txt',
 'ID17.txt',
 'AK17.txt',
 'NY17.txt',
 'OK17.txt',
 'UT17.txt',
 'DC17.txt',
 'MS17.txt']

In [26]:
# specify years for analysis
years = range(1998, 2018)

# specify rating
rating = 'DECK_COND_058'
# rating = 'SUPERSTRUCTURE_COND_059'
# rating = 'SUBSTRUCTURE_COND_060'
# rating = 'SUFFICIENCY_RATING'

# specify rating threshold
rating_threshold = 7

In [78]:
# PA
# specify filenames to read in
file_state_years = [
    '42fluna_991998-20160921081631.txt',
    '42fluna_991999-20160914090941.txt',
    '42fluna_992000-20160914070754.txt',
    '42fluna_992001-20160921070933.txt',
    '42fluna_992002-20160914140231.txt',
    '42fluna_992003-20160912070837.txt',
    '42fluna_992004-20160909192125.txt',
    '42fluna_992005-20160909182759.txt',
    '42fluna_992006-20160909132916.txt',
    '42fluna_992007-20160909121509.txt',
    '42fluna_992008-20160912090515.txt',
    '42fluna_992009-20160913064846.txt',
    'PA10.txt',
    'PA11.txt',
    'PA2012.txt',
    'PA13.txt',
    'PA14.txt',
    'PA15.txt',
    'PA16.TXT',
    'PA17.txt'
]

In [47]:
# MD
# specify filenames to read in
file_state_years = [
    '24fluna_991998-20160921081631.txt',
    '24fluna_991999-20160914090941.txt',
    '24fluna_992000-20160914070754.txt',
    '24fluna_992001-20160921070933.txt',
    '24fluna_992002-20160914140231.txt',
    '24fluna_992003-20160912070837.txt',
    '24fluna_992004-20160909192125.txt',
    '24fluna_992005-20160909182759.txt',
    '24fluna_992006-20160909132916.txt',
    '24fluna_992007-20160909121509.txt',
    '24fluna_992008-20160912090515.txt',
    '24fluna_992009-20160913064846.txt',
    'MD10.txt',
    'MD11.txt',
    'MD2012.txt',
    'MD13.txt',
    'MD14.txt',
    'MD15.txt',
    'MD16.TXT',
    'MD17.txt'
]

In [14]:
# DE
# specify filenames to read in
file_state_years = [
    '10fluna_991998-20160921081631.txt',
    '10fluna_991999-20160914090941.txt',
    '10fluna_992000-20160914070754.txt',
    '10fluna_992001-20160921070933.txt',
    '10fluna_992002-20160914140231.txt',
    '10fluna_992003-20160912070837.txt',
    '10fluna_992004-20160909192125.txt',
    '10fluna_992005-20160909182759.txt',
    '10fluna_992006-20160909132916.txt',
    '10fluna_992007-20160909121509.txt',
    '10fluna_992008-20160912090515.txt',
    '10fluna_992009-20160913064846.txt',
    'DE10.txt',
    'DE11.txt',
    'DE2012.txt',
    'DE13.txt',
    'DE14.txt',
    'DE15.txt',
    'DE16.TXT',
    'DE17.txt'
]

# Build Condition Rating Dataframe

In [80]:
# define for loop to read data for all years
df = {}

for year, file in zip(years, file_state_years):
    
    df[year] = pd.read_csv('data/nbi/' + str(year) + 'del/' + file)
    # create structure id value for index
    df[year]['ID'] = df[year]['STATE_CODE_001'].map(str) + "-" \
        + df[year]['STRUCTURE_NUMBER_008'].apply(lambda x: str(x).strip())

    # remove duplicated records for same structure id
    df[year] = df[year][~df[year]['ID'].duplicated()].copy()
    
    # add year of observation
    df[year]['year'] = year
    
    # build total list for year range
    if year == years[0]:
        df_ratings = pd.DataFrame(df[year]['ID'].copy())
        df_bridges = pd.DataFrame(df[year].copy())
    else:
        add_bridges_id = pd.DataFrame(df[year]['ID'][~df[year]['ID']
                                                         .isin(df_ratings['ID'])], copy=True)
        df_ratings = pd.concat([df_ratings, add_bridges_id], ignore_index=True, sort=False)
        add_bridges = df[year][~df[year]['ID'].isin(df_bridges['ID'])].copy()
        df_bridges = pd.concat([df_bridges, add_bridges], ignore_index=True, sort=False)
        
    # enter rating data for each year
    df_ratings = pd.merge(df_ratings, df[year]
                          .loc[:, ['ID', rating]], on='ID', how="left", sort=False)
    df_ratings.rename(columns={rating: year}, inplace=True)

/home/kevcon/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,5,11,36,37,38,39,41,44,61,63,66,67,68,69,70,75,76,77,79,80,86,87,88,98,100,102,106,107,108,109,113,114,119,120,121,123,124,128,129,130,131,133) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/kevcon/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,5,11,36,37,38,39,41,44,61,63,66,67,68,69,70,75,76,77,79,80,86,87,88,98,100,102,106,107,108,109,113,114,119,120,121,123,124,129,130,131,133) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/kevcon/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,5,11,36,37,38,39,41,44,61,63,66,67,68,69,70,75,76,77,79,80,86,87,88,98,100,102,106,107,

In [81]:
# create dataframe of 1's and 0's dependent on values above or below condition rating threshold

df_rating_change = pd.DataFrame(columns = years)

for year in years:
    df_rating_change[year] = (df_ratings.iloc[:, 
                            -len(years):][year].apply(pd.to_numeric, 
                              errors='coerce') >= rating_threshold).astype(int)

# take difference for adjacent years to show when rating drops below threshold (-1)
df_rating_change = df_rating_change.diff(axis=1)
df_rating_change['change_yr'] = ""

In [82]:
# fill column with year that rating first fell below threshold
for year in years: 
    df_rating_change['change_yr'] = np.where(df_rating_change['change_yr'] == "", 
                                           np.where(df_rating_change[year] == -1, year, ""), 
                                           df_rating_change['change_yr'])  

In [83]:
df_bridges['rating_change'] = df_rating_change['change_yr'].copy()
df_bridges = df_bridges[df_bridges['rating_change'] != ""]

In [84]:
df_bridges

,STATE_CODE_001,STRUCTURE_NUMBER_008,RECORD_TYPE_005A,ROUTE_PREFIX_005B,SERVICE_LEVEL_005C,ROUTE_NUMBER_005D,DIRECTION_005E,HIGHWAY_DISTRICT_002,COUNTY_CODE_003,PLACE_CODE_004,...,STATUS_WITH_10YR_RULE,SUFFICIENCY_ASTERC,SUFFICIENCY_RATING,STATUS_NO_10YR_RULE,ID,year,CAT10,CAT23,CAT29,rating_change
0,42,00000002140001P,1,6.0,4.0,00000,0.0,0.0,51.0,0.0,...,0,NaN,95.6,0,42-00000002140001P,1998,NaN,NaN,NaN,2001
1,42,010015001019940,1,5.0,1.0,00000,0.0,8.0,1.0,0.0,...,0,NaN,88.6,0,42-010015001019940,1998,NaN,NaN,NaN,2001
2,42,010015003000000,1,2.0,1.0,00015,1.0,8.0,1.0,0.0,...,0,NaN,92.9,0,42-010015003000000,1998,NaN,NaN,NaN,2001
4,42,010015005000000,1,2.0,1.0,00015,1.0,8.0,1.0,0.0,...,0,NaN,98.0,0,42-010015005000000,1998,NaN,NaN,NaN,2001
7,42,010015013013160,1,5.0,1.0,00000,0.0,8.0,1.0,0.0,...,0,NaN,98.0,0,42-010015013013160,1998,NaN,NaN,NaN,2001
9,42,010015015000000,1,2.0,1.0,00015,1.0,8.0,1.0,0.0,...,0,NaN,97.6,0,42-010015015000000,1998,NaN,NaN,NaN,2001
11,42,010015019000000,1,2.0,1.0,00015,1.0,8.0,1.0,0.0,...,0,NaN,93.0,0,42-010015019000000,1998,NaN,NaN,NaN,2001
12,42,010015019100000,1,2.0,1.0,00015,3.0,8.0,1.0,0.0,...,0,NaN,98.0,0,42-010015019100000,1998,NaN,NaN,NaN,2001
13,42,010015023000000,1,2.0,1.0,00015,1.0,8.0,1.0,0.0,...,0,*,96.1,0,42-010015023000000,1998,NaN,NaN,NaN,2001
14,42,010015023100000,1,2.0,1.0,00015,3.0,8.0,1.0,0.0,...,0,*,77.1,0,42-010015023100000,1998,NaN,NaN,NaN,2001


In [85]:
len(df_bridges)

16814

In [24]:
df_bridges_de

In [54]:
df_bridges_md

In [86]:
df_bridges_pa

In [88]:
df_bridges = pd.concat([df_bridges_de, df_bridges_md, df_bridges_pa], ignore_index=True)

In [89]:
len(df_bridges)

18702

# ID Parameters
- 008
    - struct number, 15-digit
- 007
    - facility carried, 18-digit
- 006A
    - feature intersected, 24-digit
- 016
    - latitude (XX deg XX min XX.XX sec), 8-digit
- 017
    - longitude (XXX deg XX min XX.XX sec), 9-digit
- 001
    - state code, 3-digit
- 003
    - county code, 3-digit

# Independent variables
Filter dataset for desired features for analysis. Items:
- 021
    - maintenance responsibility - categorical
- 027
    - year, 4-digit - feature engineer age from year as numerical
- 029
    - ADT, 6-digit - use to feature engineer ADTT value
- 109
    - ADTT (%), 2-digit - feature engineer value as numerical
- 026
    - functional classification, 2-digit - rural/urban and use
- 028A
    - lanes on struct, 2-digit - numerical
- 041
    - posting, 1-digit - categorical
- 043A
    - material, 1-digit - categorical
- 043B
    - design type, 1-digit - categorical
- 045
    - num spans, 3-digit - numerical
- 048
    - max span length (meters), 5-digit (1 decimal) - numerical
- 049
    - struct length (meters), 6-digit (1 decimal) - numerical
- 052
    - deck width (meters), 4-digit (1 decimal) - numerical
- 054B
    - min underclearance (meters), 4-digit (2 decimal) - numerical
- 061
    - channel, 1-digit - categorical
- 107
    - deck type, 1-digit - categorical
- 108A
    - wearing surface, 1-digit - categorical
- 108B
    - membrane, 1-digit - categorical
- 108C
    - deck protection, 1-digit - categorical

# Dependent variables
Bridge performance metrics. Items:
- Year of rating change

# Data Filtering

In [90]:
df_bridges.shape

(18702, 140)

## Structure Types

- 042A
    - type of service, 1-digit  
    
Filter to return observations that are highway or interchange structures (codes = 1, 6, 7 and 8)

In [91]:
df_filt = df_bridges[(df_bridges['SERVICE_ON_042A'] == 1) | (df_bridges['SERVICE_ON_042A'] == 6) | 
(df_bridges['SERVICE_ON_042A'] == 7) | (df_bridges['SERVICE_ON_042A'] == 8)]

- 043B
    - design type, 2-digit  

Filter out all observations that are culverts (code = 19)

In [92]:
df_filt = df_filt[df_filt['STRUCTURE_TYPE_043B'] != 19]

In [93]:
df_filt.shape

(15938, 140)

## Features
Return only desired features from dataset

In [94]:
# identification features

col_id = [
    'ID',
    'STRUCTURE_NUMBER_008',
    'FACILITY_CARRIED_007',
    'FEATURES_DESC_006A',
    'LAT_016',
    'LONG_017',
    'STATE_CODE_001',
    'COUNTY_CODE_003'
]

In [95]:
# independent varialbes

col_indep = [
    'year',
    'MAINTENANCE_021',
    'YEAR_BUILT_027',
    'ADT_029',
    'PERCENT_ADT_TRUCK_109',
    'FUNCTIONAL_CLASS_026',
    'TRAFFIC_LANES_ON_028A',
    'OPEN_CLOSED_POSTED_041',
    'STRUCTURE_KIND_043A', 
    'STRUCTURE_TYPE_043B',
    'MAIN_UNIT_SPANS_045',
    'MAX_SPAN_LEN_MT_048',
    'STRUCTURE_LEN_MT_049',
    'DECK_WIDTH_MT_052',
    'VERT_CLR_UND_054B',
    'CHANNEL_COND_061',
    'YEAR_RECONSTRUCTED_106',
    'DECK_STRUCTURE_TYPE_107', 
    'SURFACE_TYPE_108A', 
    'MEMBRANE_TYPE_108B',
    'DECK_PROTECTION_108C'
]

In [96]:
# dependent variables

col_dep = [
    'rating_change'
]

In [97]:
cols = col_id + col_indep + col_dep
df_filt = df_filt[cols]

In [98]:
df_filt.shape

(15938, 30)

## NaN Values

In [99]:
# return column and number of NaN values
for col in df_filt.columns:
    if df_filt[col].isnull().values.any() == True:
        print(col)
        print(sum(df_filt[col].isnull()))
    else:
        pass

LAT_016
70
LONG_017
109
COUNTY_CODE_003
4
MAINTENANCE_021
45
YEAR_BUILT_027
44
ADT_029
33
PERCENT_ADT_TRUCK_109
593
FUNCTIONAL_CLASS_026
296
TRAFFIC_LANES_ON_028A
35
OPEN_CLOSED_POSTED_041
69
STRUCTURE_KIND_043A
9
STRUCTURE_TYPE_043B
8
MAIN_UNIT_SPANS_045
57
MAX_SPAN_LEN_MT_048
6
STRUCTURE_LEN_MT_049
5
DECK_WIDTH_MT_052
52
VERT_CLR_UND_054B
109
CHANNEL_COND_061
81
YEAR_RECONSTRUCTED_106
61
DECK_STRUCTURE_TYPE_107
262
SURFACE_TYPE_108A
92
MEMBRANE_TYPE_108B
95
DECK_PROTECTION_108C
93


In [100]:
# remove any observations with NaN values
df_filt = df_filt.dropna()

In [101]:
df_filt.shape

(14617, 30)

In [102]:
df_filt.head()

,ID,STRUCTURE_NUMBER_008,FACILITY_CARRIED_007,FEATURES_DESC_006A,LAT_016,LONG_017,STATE_CODE_001,COUNTY_CODE_003,year,MAINTENANCE_021,...,STRUCTURE_LEN_MT_049,DECK_WIDTH_MT_052,VERT_CLR_UND_054B,CHANNEL_COND_061,YEAR_RECONSTRUCTED_106,DECK_STRUCTURE_TYPE_107,SURFACE_TYPE_108A,MEMBRANE_TYPE_108B,DECK_PROTECTION_108C,rating_change
5,10-1038 209,1038 209,'GRUBB-HARVEY RD ','SOUTH BRANCH NAAMANS CK ',0.0,0.0,10,3.0,1998,1.0,...,9.1,14.2,0.00,6,0.0,N,6,N,N,2008
8,10-1054A017,1054A017,'NAAMANS RD. ','NAAMANS CREEK ',0.0,0.0,10,3.0,1998,1.0,...,12.8,33.3,0.00,6,0.0,1,1,8,8,2015
19,10-1172A275,1172A275,'OLD WILMINGTON RD.','WILMINGTON NORTHERN R/R ',0.0,0.0,10,3.0,1998,1.0,...,19.8,12.4,4.77,N,0.0,N,6,N,N,2003
20,10-1177 274,1177 274,'BRACKENVILLE RD ','MILL CREEK ',0.0,0.0,10,3.0,1998,1.0,...,7.9,8.1,0.00,7,1965.0,N,6,N,N,2013
23,10-1211A011,1211A011,'KIRKWOOD HWY ','PIKE CREEK ',0.0,0.0,10,3.0,1998,1.0,...,17.1,27.3,0.00,4,1958.0,1,6,0,0,2017


# Finalize Data

In [103]:
df_fin = df_filt.copy()

## Numeric Datatypes

In [104]:
col_num = [
    'LAT_016',
    'LONG_017',
    'YEAR_BUILT_027',
    'YEAR_RECONSTRUCTED_106',
    'ADT_029',
    'PERCENT_ADT_TRUCK_109',
    'TRAFFIC_LANES_ON_028A',
    'MAIN_UNIT_SPANS_045',
    'MAX_SPAN_LEN_MT_048',
    'STRUCTURE_LEN_MT_049',
    'DECK_WIDTH_MT_052',
    'VERT_CLR_UND_054B',
    'year',
    'rating_change'
]

In [105]:
for col in col_num:
    print(df_fin[col].dtype)

float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
int64
object


In [106]:
# convert object types to int
df_fin['rating_change'] = pd.to_numeric(df_fin['rating_change'])

## Feature Engineering

### Age, Reconstruction, Downgrade
- Replace year built, year reconstructed with years since 
- Replace year of rating change with years from record until record downgrades

In [107]:
df_fin['YEAR_BUILT_027'] = df_fin['year'] - df_fin['YEAR_BUILT_027']
df_fin.rename(columns={'YEAR_BUILT_027': 'AGE'}, inplace=True)

In [108]:
df_fin['YEARS_SINCE_RECON'] = df_fin.apply(lambda x: x['AGE'] 
             if x['YEAR_RECONSTRUCTED_106'] == 0 else x['year'] - x['YEAR_RECONSTRUCTED_106'], axis=1);

In [109]:
df_fin['TO_DWNGRD'] = df_fin['rating_change'] - df_fin['year']

### ADTT
Replace percentage of ADT trucks with ADTT value

In [110]:
df_fin['PERCENT_ADT_TRUCK_109'] = df_fin['PERCENT_ADT_TRUCK_109'] * df_fin['ADT_029']
df_fin.rename(columns={'PERCENT_ADT_TRUCK_109': 'ADTT'}, inplace=True)

### Location

In [31]:
# may use later
# calc distance using Lat and Long values

## Define numerical feature dataframe

In [111]:
df_num = df_fin[[
    'ID',
    'AGE',
    'YEARS_SINCE_RECON',
    'ADT_029',
    'ADTT',
    'TRAFFIC_LANES_ON_028A',
    'MAIN_UNIT_SPANS_045',
    'MAX_SPAN_LEN_MT_048',
    'STRUCTURE_LEN_MT_049',
    'DECK_WIDTH_MT_052',
    'VERT_CLR_UND_054B'
]]

In [112]:
df_num.shape

(14617, 11)

In [113]:
df_num.head()

,ID,AGE,YEARS_SINCE_RECON,ADT_029,ADTT,TRAFFIC_LANES_ON_028A,MAIN_UNIT_SPANS_045,MAX_SPAN_LEN_MT_048,STRUCTURE_LEN_MT_049,DECK_WIDTH_MT_052,VERT_CLR_UND_054B
5,10-1038 209,4.0,4.0,13090.0,39270.0,2.0,1.0,8.2,9.1,14.2,0.00
8,10-1054A017,6.0,6.0,14112.0,141120.0,5.0,1.0,12.2,12.8,33.3,0.00
19,10-1172A275,11.0,11.0,1572.0,11004.0,2.0,3.0,6.1,19.8,12.4,4.77
20,10-1177 274,98.0,33.0,6580.0,46060.0,2.0,1.0,6.1,7.9,8.1,0.00
23,10-1211A011,50.0,40.0,37376.0,112128.0,4.0,1.0,15.8,17.1,27.3,0.00


## Categorical Values

In [114]:
col_cat = [
    'MAINTENANCE_021',
    'FUNCTIONAL_CLASS_026',
    'OPEN_CLOSED_POSTED_041',
    'STRUCTURE_KIND_043A', 
    'STRUCTURE_TYPE_043B',
    'CHANNEL_COND_061',
    'DECK_STRUCTURE_TYPE_107', 
    'SURFACE_TYPE_108A', 
    'MEMBRANE_TYPE_108B',
    'DECK_PROTECTION_108C'
]

In [115]:
# examine categorical values
for col in col_cat:
    print(col)
    print(df_fin[col].unique())

MAINTENANCE_021
[ 1. 31. 32. 27. 70. 60. 21.  2. 66.  4. 12. 73. 74.  3. 11. 26. 80. 25.
 71.]
FUNCTIONAL_CLASS_026
[17. 14.  9. 19. 16.  7.  2. 11.  6.  8. 12.  1.]
OPEN_CLOSED_POSTED_041
['A' 'P' 'R' 'K' 'B' 'E' 'G' 'D']
STRUCTURE_KIND_043A
[1. 5. 7. 8. 3. 4. 2. 6. 0. 9.]
STRUCTURE_TYPE_043B
[ 7.  5.  1. 11.  2. 10. 13.  3. 14.  4.  0.  9. 16. 12.  6. 20. 22. 21.]
CHANNEL_COND_061
['6' 'N' '7' '4' '8' '5' '9' '3' '0' '2']
DECK_STRUCTURE_TYPE_107
['N' '1' '7' '2' '8' '9' '5' '3' '6' '4']
SURFACE_TYPE_108A
['6' '1' '8' '0' '3' 'N' '7' '2' '9' '5' '4']
MEMBRANE_TYPE_108B
['N' '8' '0' '3' '9' '2' '1']
DECK_PROTECTION_108C
['N' '8' '0' '1' '9' '2' '4' '7' '6' '3']


In [116]:
df_cat = df_fin[col_cat]

In [117]:
df_cat.shape

(14617, 10)

In [118]:
cats = df_cat.shape[1]

In [119]:
df_cat.head()

,MAINTENANCE_021,FUNCTIONAL_CLASS_026,OPEN_CLOSED_POSTED_041,STRUCTURE_KIND_043A,STRUCTURE_TYPE_043B,CHANNEL_COND_061,DECK_STRUCTURE_TYPE_107,SURFACE_TYPE_108A,MEMBRANE_TYPE_108B,DECK_PROTECTION_108C
5,1.0,17.0,A,1.0,7.0,6,N,6,N,N
8,1.0,14.0,A,5.0,5.0,6,1,1,8,8
19,1.0,9.0,A,7.0,1.0,N,N,6,N,N
20,1.0,17.0,A,8.0,11.0,7,N,6,N,N
23,1.0,14.0,A,3.0,2.0,4,1,6,0,0


## Define categorical feature dataframe

In [120]:
# create sparse matrix for categorical values and add to feature matrix
for col in col_cat:
    cat_df = pd.get_dummies(df_cat[col], prefix=col)
    df_cat = df_cat.join(cat_df)

df_cat = df_cat.iloc[:, cats:]

In [121]:
df_cat

,MAINTENANCE_021_1.0,MAINTENANCE_021_2.0,MAINTENANCE_021_3.0,MAINTENANCE_021_4.0,MAINTENANCE_021_11.0,MAINTENANCE_021_12.0,MAINTENANCE_021_21.0,MAINTENANCE_021_25.0,MAINTENANCE_021_26.0,MAINTENANCE_021_27.0,...,DECK_PROTECTION_108C_0,DECK_PROTECTION_108C_1,DECK_PROTECTION_108C_2,DECK_PROTECTION_108C_3,DECK_PROTECTION_108C_4,DECK_PROTECTION_108C_6,DECK_PROTECTION_108C_7,DECK_PROTECTION_108C_8,DECK_PROTECTION_108C_9,DECK_PROTECTION_108C_N
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
19,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
20,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
23,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
25,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
26,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
27,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
32,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
36,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [122]:
# insert identification code
df_cat.insert(0, 'ID', df_bridges['ID'])

In [123]:
df_cat.head()

,ID,MAINTENANCE_021_1.0,MAINTENANCE_021_2.0,MAINTENANCE_021_3.0,MAINTENANCE_021_4.0,MAINTENANCE_021_11.0,MAINTENANCE_021_12.0,MAINTENANCE_021_21.0,MAINTENANCE_021_25.0,MAINTENANCE_021_26.0,...,DECK_PROTECTION_108C_0,DECK_PROTECTION_108C_1,DECK_PROTECTION_108C_2,DECK_PROTECTION_108C_3,DECK_PROTECTION_108C_4,DECK_PROTECTION_108C_6,DECK_PROTECTION_108C_7,DECK_PROTECTION_108C_8,DECK_PROTECTION_108C_9,DECK_PROTECTION_108C_N
5,10-1038 209,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,10-1054A017,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
19,10-1172A275,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
20,10-1177 274,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
23,10-1211A011,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


## Define target feature dataframe

In [124]:
Y = df_fin[[
    'TO_DWNGRD'
]]

In [125]:
# insert identification code
Y.insert(0, 'ID', df_bridges['ID'])

In [126]:
Y.head()

,ID,TO_DWNGRD
5,10-1038 209,10
8,10-1054A017,17
19,10-1172A275,5
20,10-1177 274,15
23,10-1211A011,19


# Output data to file

In [127]:
# ouput numeric data
df_num.to_csv('data/data_numeric.csv', index=False)

In [128]:
# output categorical data
df_cat.to_csv('data/data_categorical.csv', index=False)

In [129]:
# output target varialbes
Y.to_csv('data/data_targets.csv', index=False)